# Setup codes

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError


import ipywidgets as ipw
from IPython.display import display, clear_output

In [ ]:
compname = 'quantum-mobile'
qe_version = '6.2.1'

In [ ]:
def setup_codes(computer_name):
    computer = Computer.get(computer_name)
    
    qe_path = "/home/ubuntu/codes/q-e-qe-6.2.1-fda42b8045f604ef8cc8400dac1b2a06cb3f6e91/bin/"
    cp2k_path = "/home/ubuntu/codes/cp2k-5.1/exe/"
    
    # CP2K
    cp2k_version = '6.0'
    cp2k_revision = '18265'
    code_label = "cp2k_{}_{}".format(cp2k_version, cp2k_revision)
    code_full_name = "{}@{}".format(code_label, computer_name)
    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, cp2k_path + "cp2k.ssmp"))
        code.label = code_label
        code.description = "CP2K Code"
        code.set_input_plugin_name("cp2k")
        prepend_text  = "### code prepend_text start ###\n"
        prepend_text += "module load CP2K\n"
        prepend_text += "export CP2K_DATA_DIR=/home/ubuntu/codes/cp2k-5.1/\n"
        prepend_text += "### code prepend_text end ###\n"
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        code._reveal()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))        
    
    # QuantumESPRESSO pw
    for codename in ['pw', 'pp', 'ph', 'projwfc', 'matdyn', 'q2r']:
        code_label = "{}-{}".format(codename, qe_version)
        code_full_name = "{}@{}".format(code_label, computer_name)
        try:
            Code.get_from_string(code_full_name)
        except (NotExistent, MultipleObjectsError):            
            code = Code(remote_computer_exec=(computer, qe_path + "{}.x".format(codename)))
            code.label = code_label
            code.description = "Quantum ESPRESSO {}.x version {}".format(codename, qe_version)
            code.set_input_plugin_name("quantumespresso.{}".format(codename))
            prepend_text  = "### code prepend_text start ###\n"
            prepend_text += "module load QuantumESPRESSO/6.1.0-CrayIntel-17.08\n"
            prepend_text += "### code prepend_text end ###\n"
            code.set_prepend_text(prepend_text)
            code.set_append_text("")
            code.store()
            code._reveal()
            ! verdi code show "{code_full_name}"
        else:
            print("Code '{}' already existent".format(code_full_name))

In [ ]:
def on_setup_codes(b):
    with setup_code_out:
        clear_output()
        setup_codes(compname)

setup_code_out = ipw.Output()
btn_setup_codes = ipw.Button(description="Setup Codes")
btn_setup_codes.on_click(on_setup_codes)
display(btn_setup_codes, setup_code_out)